In [149]:
import os
import pandas 
import numpy as np
import matplotlib.pyplot as plt

In [150]:
def res_file_to_df(file):
    '''
    Converts a res file into a pandas DataFrame for a single day
    '''
    # header information
    columns = ['MJD', 
               'frac of a day', 
               'GPS range AB [m]', 
               'GPS range rate AB [m/s]', 
               'Kband range [m]',
               'Kband range rate [m/s]',
               'O-C range rate [m/s]',
               'Latitude [deg]',
               'Longitude [deg]',
               'Arg. of lat. [deg]',
               'beta [deg]']
    # collect data
    data_array = [] # empty container 
    with open(path) as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if i < 12: continue # skip header information
            line_entries = line.split(' ') # split lines with space delimiter
            line_entries = np.array(line_entries) # convert to numpy array
            line_entries = np.delete(line_entries, np.argwhere(line_entries=='')).astype(float) # convert to floats
            data_array.append(line_entries) # collect vectors
    data_array = np.vstack(data_array) # stack vectors into an array 
    df = pandas.DataFrame(data=data_array,columns=columns) # convert stack to pandas DataFrame
    return df

In [140]:
path = '/Users/brandonlpanos/Desktop/grace/datasets/2019-01/KBRpfres_op_19003.res'
df = res_file_to_df(path)
df.head()

,MJD,frac of a day,GPS range AB [m],GPS range rate AB [m/s],Kband range [m],Kband range rate [m/s],O-C range rate [m/s],Latitude [deg],Longitude [deg],Arg. of lat. [deg],beta [deg]
0,58486.0,0.000000,193483.244640,0.184121,78448.581005,0.184121,-9.513300e-08,7.771713,106.802600,7.863474,-63.426621
1,58486.0,0.000058,193484.164665,0.183889,78449.501029,0.183889,-1.758749e-07,8.088374,106.787365,8.180184,-63.426671
2,58486.0,0.000116,193485.083539,0.183660,78450.419902,0.183660,-1.197775e-08,8.405043,106.772139,8.496903,-63.426721
3,58486.0,0.000174,193486.001275,0.183435,78451.337638,0.183434,-1.766628e-07,8.721722,106.756923,8.813631,-63.426771
4,58486.0,0.000231,193486.917889,0.183211,78452.254251,0.183211,6.985879e-08,9.038410,106.741716,9.130369,-63.426820


In [160]:
y = np.array(df['O-C range rate [m/s]']) # target random variable risidules 
df2 = df.copy()
df2 = df2.drop(['O-C range rate [m/s]'], axis=1)
X = df2.to_numpy()
X.shape

(17280, 10)

## Calculate the MI  
Setup for the MINE-network  
Use the post fit residuals (7th column O-C range rate [m/s]) as one random variable  
The other random variable is a vector of what remains. This vector gives the position of the satellite over earth with some redundant infomation, like different coordinates and time stamps.  
For the upper channel we feed in vectors and residuals as they appear in the tables  
For the lower channel we shuffle the residuals  
The network will then try and distinguish between residuals tied to actual positions and those that are random   
If the network can distinguish then the position is important, i.e., some signal has been consumed into the residuals

In [163]:
# create datasets by concatenating data across all days from all 12 months
X = None
y = None
root_path  = '/Users/brandonlpanos/Desktop/grace/datasets/'
for month in os.listdir(root_path):    
    for day in os.listdir(root_path + month):
        path = root_path + month + '/' + day
        
        try: df = res_file_to_df(path) # convert res file into pandas DataFrame
        except: continue # some datasets are empty, if any problems just skip
            
        y_day = np.array(df['O-C range rate [m/s]']) # target random variable residuals 
        df2 = df.copy()
        df2 = df2.drop(['O-C range rate [m/s]'], axis=1) # drop target variable from df
        X_day = df2.to_numpy() # construct matrix out of remaining columns
        
        try: X = np.concatenate( (X, X_day), axis=0 )
        except: X = X_day
        
        try: y = np.concatenate( (y, y_day) )
        except: y = y_day
        
        print(X.shape, y.shape) 

(17280, 10) (17280,)
(34441, 10) (34441,)
(51721, 10) (51721,)
(69001, 10) (69001,)
(86281, 10) (86281,)
(103561, 10) (103561,)
(120841, 10) (120841,)
(138121, 10) (138121,)
(155401, 10) (155401,)
(172607, 10) (172607,)
(189887, 10) (189887,)
(207167, 10) (207167,)
(224447, 10) (224447,)
(241727, 10) (241727,)
(259007, 10) (259007,)
(276287, 10) (276287,)
(293567, 10) (293567,)
(310847, 10) (310847,)
(328127, 10) (328127,)
(345407, 10) (345407,)
(362687, 10) (362687,)
(379967, 10) (379967,)
(397247, 10) (397247,)
(414527, 10) (414527,)
(431725, 10) (431725,)
(449005, 10) (449005,)
(466285, 10) (466285,)
(483565, 10) (483565,)
(500845, 10) (500845,)
(518125, 10) (518125,)
(535405, 10) (535405,)
(552685, 10) (552685,)
(569965, 10) (569965,)
(587245, 10) (587245,)
(604525, 10) (604525,)
(621805, 10) (621805,)
(639085, 10) (639085,)
(656365, 10) (656365,)
(673645, 10) (673645,)
(690925, 10) (690925,)
(708205, 10) (708205,)
(725378, 10) (725378,)
(742658, 10) (742658,)
(759938, 10) (759938,

(5757311, 10) (5757311,)
(5774495, 10) (5774495,)
(5791775, 10) (5791775,)
(5809055, 10) (5809055,)
(5826335, 10) (5826335,)
(5843615, 10) (5843615,)
(5860895, 10) (5860895,)
(5878175, 10) (5878175,)
(5895455, 10) (5895455,)
(5912646, 10) (5912646,)
(5929926, 10) (5929926,)
(5947067, 10) (5947067,)
(5964347, 10) (5964347,)
(5981627, 10) (5981627,)
